In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (20,10)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
home = pd.read_csv('../input/bengaluru-house-price-data/Bengaluru_House_Data.csv')
home.head()

In [ ]:
home2 = home.copy()

In [ ]:
home.info()

In [ ]:
#Checking for count of each categorical columns
def count(df):
    for val in df.columns:
        print(df[val].value_counts())
        print("*************************")

count(home)       

In [ ]:
home.isna().sum()

In [ ]:
#assuming that these are not a factor for price
del_col = ['area_type','availability','society','balcony']
home = home.drop(del_col, axis=1)

In [ ]:
#checking the mean and median for numerical variables
home.describe()

In [ ]:
#Imputing columns value based on above data
home['location'] = home['location'].fillna('Sarjapur Road')
home['size'] = home['size'].fillna('2 BHK')
home['bath'] = home['bath'].fillna(home['bath'].median())


In [ ]:
#checking the datatype of variables
home.info()

In [ ]:
home.shape

In [ ]:
home['bhk'] = home['size'].apply(lambda x : int(x.split(' ')[0]))

In [ ]:
home.bhk.unique()

In [ ]:
home[home.bhk > 20]

In [ ]:
home.total_sqft.unique()

In [ ]:
def isFloat(x):
    try:
        float(x)
    except:
        return False
    return True

In [ ]:
home[~home['total_sqft'].apply(isFloat)]

In [ ]:
def convertRange(x):
    temp = x.split('-')
    if len(temp) == 2:
        return (float(temp[0]) + float(temp[1]))/2
    try:
        return float(x)
    except:
        return None

In [ ]:
home3 = home.copy()

In [ ]:
home3['total_sqft'] = home3['total_sqft'].apply(convertRange)
home3.head()

### Feature Engineering(Narrowing the Ranges)

In [ ]:
home4 = home3.copy()
home4['price_per_sqft'] = (home4.price * 100000) / home4.total_sqft
home4.head()

In [ ]:
len(home4.location.unique())

In [ ]:
home4.location = home4.location.apply(lambda x: x.strip())
location_count = home4.location.value_counts()
location_count

In [ ]:
location_count_less_10 = location_count[location_count <= 10]
location_count_less_10

In [ ]:
home4.location = home4.location.apply(lambda x : 'other' if x in location_count_less_10 else x)


In [ ]:
home4.head()

### Outlier Detection and Removal

In [ ]:
home4[home4.total_sqft/home4.bhk < 300].head()

In [ ]:
home4.shape

In [ ]:
home6 = home4[~(home4.total_sqft/home4.bhk < 300)]
home6.shape

In [ ]:
home6.price_per_sqft.describe()

In [ ]:
def remove_outliers_sqft(df):
    df_output = pd.DataFrame()
    for key,subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sqft)
        st = np.std(subdf.price_per_sqft)
        gen_df = subdf[(subdf.price_per_sqft > (m-st)) & (subdf.price_per_sqft <= (m+st))]
        df_output = pd.concat([df_output,gen_df],ignore_index =True)
    return df_output
home7 = remove_outliers_sqft(home6)
home7.shape

In [ ]:
def plot_scatter_chart(df,location):
    bhk2 = df[(df.location == location) & (df.bhk == 2)]
    bhk3 = df[(df.location == location) & (df.bhk == 3)]
    matplotlib.rcParams['figure.figsize'] = (20,10)
    plt.scatter(bhk2.total_sqft,bhk2.price,color='red',label='2 BHK',s=50)
    plt.scatter(bhk3.total_sqft,bhk3.price,marker='D',color='green',label='3 BHK',s=50)
    plt.xlabel('Total square Feet area')
    plt.ylabel('Price ')
    plt.title(location)
    plt.legend()

plot_scatter_chart(home7,'Rajaji Nagar')    

In [ ]:
def bhk_outlier_remover(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df.price_per_sqft),
                'std': np.std(bhk_df.price_per_sqft),
                'count': bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('bhk'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')

home8 = bhk_outlier_remover(home7)
home8.shape

In [ ]:
plot_scatter_chart(home8,'Rajaji Nagar')    

In [ ]:

plt.hist(home8.price_per_sqft,rwidth=0.8)
plt.xlabel("Price Per Square Feet")
plt.ylabel("Count")

In [ ]:
home8.bath.unique()

In [ ]:
plt.hist(home8.bath,rwidth=0.8)
plt.xlabel("Number of bathrooms")
plt.ylabel("Count")

Series' object has no attribute 'unqiue'

In [ ]:
home9 = home8[home8.bath<home8.bhk+2]
home9.shape

In [ ]:
home9.head()

In [ ]:
home10 = home9.drop(['size','price_per_sqft'],axis='columns')
home10.head()

In [ ]:
home10.bath.value_counts()

In [ ]:
dummies = pd.get_dummies(home10.location,drop_first=True)
dummies

In [ ]:
home11 = pd.concat([home10,dummies],axis='columns')
home11.head(3)

In [ ]:
home12 = home11.drop('location',axis='columns')
home12.head()

In [ ]:
home12.shape

In [ ]:
X = home12.drop('price',axis='columns')
X.head()

In [ ]:
y = home12.price
y.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=10)

In [ ]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train,y_train)
linreg.score(X_test,y_test)

## Use K Fold cross validation to measure accuracy of our LinearRegression model

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

cross_val_score(LinearRegression(), X, y, cv=cv)



# Find best model using GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

def find_best_model_using_gridsearchcv(X,y):
    algorithms = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algorithms.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

find_best_model_using_gridsearchcv(X,y)

In [ ]:
def predict_price(location,sqft,bath,bhk):    
    loc_index = np.where(X.columns==location)[0][0]

    x = np.zeros(len(X.columns))
    x[0] = sqft
    x[1] = bath
    x[2] = bhk
    if loc_index >= 0:
        x[loc_index] = 1

    return linreg.predict([x])[0]

In [ ]:
predict_price('1st Phase JP Nagar',1000, 2, 2)

## Exporting the tested model to a pickle file


In [ ]:
import pickle
with open('banglore_home_prices_model.pickle','wb') as f:
    pickle.dump(linreg,f)


In [ ]:
import json
columns = {
    'data_columns' : [col.lower() for col in X.columns]
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))


In [ ]:
from django.contrib import admin
from django.contrib import admin
